### Importing Libraries 

In [1]:
from nltk.tokenize import word_tokenize
import nltk
import re
import gensim
import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb
import keras.backend as K
from itertools import chain
from keras.utils import to_categorical
from keras.callbacks import *
from keras.optimizers import *
import matplotlib.pyplot as plt
from keras.initializers import *
from scipy.stats import pearsonr
from sklearn.decomposition import PCA
from keras.engine.topology import Layer
from keras.models import Sequential,Model
from keras.layers.embeddings import Embedding
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.wrappers.scikit_learn import KerasRegressor
from keras.preprocessing.sequence import pad_sequences
from keras import initializers, regularizers, constraints
from sklearn.model_selection import train_test_split
from keras.layers import Input, Dense, Dropout, AveragePooling1D, LSTM, Conv1D, GlobalAveragePooling1D, GlobalMaxPooling1D, Flatten, GRU, Bidirectional
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.preprocessing import sequence
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
#from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score,precision_score
import matplotlib.pyplot as plt
from keras.layers import concatenate,dot
from keras.preprocessing import sequence
from keras.utils.vis_utils import model_to_dot
from keras.utils.vis_utils import plot_model
#from skmultilearn.problem_transform import LabelPowerset

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
Using TensorFlow backend.


### Monting Google Drive **folders**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


### Files **Reading**

In [3]:
train=pd.read_csv("/content/drive/My Drive/Project with Enas Kh/Thesis/Codes and Dataset/ITP_Data/current experiment/train.csv")
test = pd.read_csv("/content/drive/My Drive/Project with Enas Kh/Thesis/Codes and Dataset/ITP_Data/current experiment/test.csv")
trainsample= train.drop(['id'],axis=1)
testsample= test.drop(['id'],axis=1)

### Showing the file content (After dropping the ID coulmn)


In [4]:
trainsample

,sex,bw_6,bw_12,bw_18,bw_24,dead,age(days),target
0,0,36,39.0,47.0,49.0,1,844,304
1,0,33,36.0,41.0,66.0,1,975,435
2,0,34,51.0,53.0,0.0,1,721,181
3,0,28,33.0,35.0,0.0,1,609,69
4,0,31,43.0,41.0,35.0,1,969,429
...,...,...,...,...,...,...,...,...
1393,1,36,33.0,40.1,40.0,1,1083,543
1394,1,27,42.1,45.0,0.0,1,373,-167
1395,1,43,41.9,45.4,47.2,1,903,363
1396,1,42,45.4,41.6,44.6,1,813,273


In [5]:
testsample

,sex,bw_6,bw_12,bw_18,bw_24,dead,age(days),target
0,0,29,30,33,39,1,918,378
1,0,29,25,30,34,1,785,245
2,0,20,23,25,28,1,840,300
3,0,24,25,29,36,1,675,135
4,0,32,33,39,40,1,572,32
...,...,...,...,...,...,...,...,...
345,1,42,44,43,38,1,770,230
346,1,41,44,46,43,1,1130,590
347,0,25,27,32,33,1,1158,618
348,0,32,40,46,43,1,1069,529


### Display new class counts

In [6]:
trainsample.target.value_counts().sort_index()
#x=sample.loc[:, sample.columns != 'target']
train, test = train_test_split(trainsample, test_size=.33, random_state=7,shuffle=True)
#y_train = to_categorical(train.target,num_classes=3)
#y_test = to_categorical(test.target,num_classes=3)
y_train =train.target
y_test =test.target

### Showing the rows and columns shape

In [7]:
train.shape, test.shape, y_train.shape, y_test.shape

((936, 8), (462, 8), (936,), (462,))

In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score,precision_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_predict

In [9]:
train_attributes=train.columns.values[2:202]
train_correlations=train[train_attributes].corr().abs().unstack().sort_values(kind='quicksort').reset_index()
train_correlations=train_correlations[train_correlations['level_0']!=train_correlations['level_1']]
train_correlations.head(10)

,level_0,level_1,0
0,age(days),bw_12,0.083455
1,target,bw_12,0.083455
2,bw_12,target,0.083455
3,bw_12,age(days),0.083455
4,bw_24,bw_12,0.100154
5,bw_12,bw_24,0.100154
6,dead,bw_12,0.167673
7,bw_12,dead,0.167673
8,dead,bw_18,0.249879
9,bw_18,dead,0.249879


### Gradient Boosting with DTC

In [10]:
gradientBoosting = RandomForestClassifier(criterion = "gini", 
                                       max_depth= 6,
                                       n_estimators=100, 
                                       max_features='log2', 
                                       )
gradientBoosting.fit(train, y_train)
Y_pred = gradientBoosting.predict(test)
print(classification_report(y_test,Y_pred ))

              precision    recall  f1-score   support

        -357       0.00      0.00      0.00         0
        -328       0.00      0.00      0.00         0
        -326       0.00      0.00      0.00         1
        -324       0.00      0.00      0.00         1
        -316       0.00      0.00      0.00         1
        -314       0.00      0.00      0.00         1
        -307       0.00      0.00      0.00         0
        -298       0.00      0.00      0.00         2
        -248       0.00      0.00      0.00         0
        -244       0.00      0.00      0.00         1
        -241       0.00      0.00      0.00         1
        -236       0.00      0.00      0.00         0
        -231       0.00      0.00      0.00         1
        -222       0.00      0.00      0.00         1
        -214       0.25      1.00      0.40         1
        -213       0.00      0.00      0.00         1
        -200       0.00      0.00      0.00         0
        -196       0.00    

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Gradient Boosting Classifier


In [11]:
gradientBoosting = GradientBoostingClassifier()
gradientBoosting.fit(train, y_train)
Y_pred = gradientBoosting.predict(test)
print(classification_report(y_test,Y_pred ))

              precision    recall  f1-score   support

        -326       0.00      0.00      0.00         1
        -324       0.00      0.00      0.00         1
        -316       0.00      0.00      0.00         1
        -314       0.00      0.00      0.00         1
        -298       0.00      0.00      0.00         2
        -244       0.00      0.00      0.00         1
        -241       0.00      0.00      0.00         1
        -231       0.00      0.00      0.00         1
        -222       0.00      0.00      0.00         1
        -214       0.00      0.00      0.00         1
        -213       0.00      0.00      0.00         1
        -196       0.00      0.00      0.00         1
        -195       0.00      0.00      0.00         0
        -186       0.00      0.00      0.00         1
        -185       0.00      0.00      0.00         1
        -183       0.00      0.00      0.00         1
        -177       0.00      0.00      0.00         1
        -169       0.00    

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Support Vector Machine 

In [12]:
svm_clf1 = SVC()
svm_clf1.fit(train, y_train)
Y_pred = svm_clf1.predict(test)
print(classification_report(y_test,Y_pred ))

              precision    recall  f1-score   support

        -326       0.00      0.00      0.00         1
        -324       0.00      0.00      0.00         1
        -316       0.00      0.00      0.00         1
        -314       0.00      0.00      0.00         1
        -298       0.00      0.00      0.00         2
        -244       0.00      0.00      0.00         1
        -241       0.00      0.00      0.00         1
        -231       0.00      0.00      0.00         1
        -222       0.00      0.00      0.00         1
        -214       0.00      0.00      0.00         1
        -213       0.00      0.00      0.00         1
        -196       0.00      0.00      0.00         1
        -186       0.00      0.00      0.00         1
        -185       0.03      1.00      0.06         1
        -183       0.00      0.00      0.00         1
        -177       0.00      0.00      0.00         1
        -169       0.00      0.00      0.00         1
        -164       0.00    

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### SVM with Linear kernel

In [13]:
svm_clf2 = SVC(kernel='linear', C=10)
svm_clf2.fit(train, y_train)
Y_pred = svm_clf2.predict(test)
print(classification_report(y_test,Y_pred ))

              precision    recall  f1-score   support

        -328       0.00      0.00      0.00         0
        -326       0.00      0.00      0.00         1
        -324       0.00      0.00      0.00         1
        -316       0.00      0.00      0.00         1
        -314       0.00      0.00      0.00         1
        -307       0.00      0.00      0.00         0
        -298       1.00      0.50      0.67         2
        -244       0.00      0.00      0.00         1
        -241       0.00      0.00      0.00         1
        -240       0.00      0.00      0.00         0
        -236       0.00      0.00      0.00         0
        -232       0.00      0.00      0.00         0
        -231       0.00      0.00      0.00         1
        -222       0.00      0.00      0.00         1
        -215       0.00      0.00      0.00         0
        -214       0.50      1.00      0.67         1
        -213       0.00      0.00      0.00         1
        -199       0.00    

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


SVM with Plynomial kernel

In [14]:
poly_kernel_svm_clf = SVC(kernel="poly", degree=3, coef0=1, C=5)
poly_kernel_svm_clf.fit(train, y_train)
Y_pred = poly_kernel_svm_clf.predict(test)
print(classification_report(y_test,Y_pred ))

              precision    recall  f1-score   support

        -328       0.00      0.00      0.00         0
        -326       0.00      0.00      0.00         1
        -324       0.00      0.00      0.00         1
        -316       0.00      0.00      0.00         1
        -314       0.00      0.00      0.00         1
        -307       0.00      0.00      0.00         0
        -298       0.00      0.00      0.00         2
        -244       0.00      0.00      0.00         1
        -241       0.00      0.00      0.00         1
        -231       0.00      0.00      0.00         1
        -222       0.00      0.00      0.00         1
        -214       0.00      0.00      0.00         1
        -213       0.00      0.00      0.00         1
        -196       0.00      0.00      0.00         1
        -186       0.00      0.00      0.00         1
        -185       0.05      1.00      0.09         1
        -183       0.00      0.00      0.00         1
        -177       0.00    

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### SVM with RBF kernel

In [15]:
rbf_kernel_svm_clf = SVC(kernel="rbf", gamma=0.1, C=1000)
rbf_kernel_svm_clf.fit(train, y_train)
Y_pred = rbf_kernel_svm_clf.predict(test)
print(classification_report(y_test,Y_pred ))

              precision    recall  f1-score   support

        -326       0.00      0.00      0.00         1
        -324       0.00      0.00      0.00         1
        -316       0.00      0.00      0.00         1
        -314       0.00      0.00      0.00         1
        -298       0.00      0.00      0.00         2
        -244       0.00      0.00      0.00         1
        -241       0.00      0.00      0.00         1
        -231       0.00      0.00      0.00         1
        -222       0.00      0.00      0.00         1
        -214       0.00      0.00      0.00         1
        -213       0.00      0.00      0.00         1
        -196       0.00      0.00      0.00         1
        -186       0.00      0.00      0.00         1
        -185       1.00      1.00      1.00         1
        -183       0.00      0.00      0.00         1
        -177       0.00      0.00      0.00         1
        -169       0.00      0.00      0.00         1
        -164       0.00    

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Logistic Regression

In [16]:
logreg = LogisticRegression()
logreg.fit(train, y_train)
Y_pred = logreg.predict(test)
print(classification_report(y_test,Y_pred ))

              precision    recall  f1-score   support

        -357       0.00      0.00      0.00         0
        -326       0.00      0.00      0.00         1
        -324       0.00      0.00      0.00         1
        -319       0.00      0.00      0.00         0
        -316       0.00      0.00      0.00         1
        -314       0.00      0.00      0.00         1
        -307       0.00      0.00      0.00         0
        -298       0.00      0.00      0.00         2
        -244       0.00      0.00      0.00         1
        -241       0.00      0.00      0.00         1
        -239       0.00      0.00      0.00         0
        -236       0.00      0.00      0.00         0
        -231       0.00      0.00      0.00         1
        -222       0.00      0.00      0.00         1
        -214       0.25      1.00      0.40         1
        -213       0.00      0.00      0.00         1
        -199       0.00      0.00      0.00         0
        -196       0.00    

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in

### GaussianNB

In [17]:
gaussian = GaussianNB()
gaussian.fit(train, y_train)
Y_pred = gaussian.predict(test)
print(classification_report(y_test,Y_pred ))

              precision    recall  f1-score   support

        -328       0.00      0.00      0.00         0
        -326       0.00      0.00      0.00         1
        -324       0.00      0.00      0.00         1
        -316       0.00      0.00      0.00         1
        -314       0.00      0.00      0.00         1
        -307       0.00      0.00      0.00         0
        -298       1.00      0.50      0.67         2
        -244       0.00      0.00      0.00         1
        -241       0.00      0.00      0.00         1
        -240       0.00      0.00      0.00         0
        -236       0.00      0.00      0.00         0
        -232       0.00      0.00      0.00         0
        -231       0.00      0.00      0.00         1
        -222       0.00      0.00      0.00         1
        -215       0.00      0.00      0.00         0
        -214       1.00      1.00      1.00         1
        -213       1.00      1.00      1.00         1
        -200       0.00    

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Decision Tree (default parameters)

In [18]:
decision_tree1 = DecisionTreeClassifier()
decision_tree1.fit(train, y_train)
Y_pred = decision_tree1.predict(test)
print(classification_report(y_test,Y_pred ))

              precision    recall  f1-score   support

        -328       0.00      0.00      0.00         0
        -326       0.00      0.00      0.00         1
        -324       0.00      0.00      0.00         1
        -319       0.00      0.00      0.00         0
        -316       0.00      0.00      0.00         1
        -314       0.00      0.00      0.00         1
        -298       0.00      0.00      0.00         2
        -248       0.00      0.00      0.00         0
        -244       0.00      0.00      0.00         1
        -241       0.00      0.00      0.00         1
        -239       0.00      0.00      0.00         0
        -236       0.00      0.00      0.00         0
        -232       0.00      0.00      0.00         0
        -231       0.00      0.00      0.00         1
        -222       0.00      0.00      0.00         1
        -215       0.00      0.00      0.00         0
        -214       0.50      1.00      0.67         1
        -213       0.00    

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Decision tree 2

In [19]:
decision_tree2 = DecisionTreeClassifier(max_depth=3, min_samples_split=0.1, min_samples_leaf=0.1)
decision_tree2.fit(train, y_train)
Y_pred = decision_tree2.predict(test)
print(classification_report(y_test,Y_pred ))

              precision    recall  f1-score   support

        -326       0.00      0.00      0.00         1
        -324       0.00      0.00      0.00         1
        -316       0.00      0.00      0.00         1
        -314       0.00      0.00      0.00         1
        -298       0.00      0.00      0.00         2
        -244       0.00      0.00      0.00         1
        -241       0.00      0.00      0.00         1
        -231       0.00      0.00      0.00         1
        -222       0.00      0.00      0.00         1
        -214       0.00      0.00      0.00         1
        -213       0.00      0.00      0.00         1
        -196       0.00      0.00      0.00         1
        -186       0.00      0.00      0.00         1
        -185       0.01      1.00      0.01         1
        -183       0.00      0.00      0.00         1
        -177       0.00      0.00      0.00         1
        -169       0.00      0.00      0.00         1
        -164       0.00    

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Decision tree 3 

In [20]:
decision_tree3 = DecisionTreeClassifier(max_depth=10, min_samples_split=3,min_samples_leaf=5) 
decision_tree3.fit(train, y_train)
Y_pred = decision_tree3.predict(test)
print(classification_report(y_test,Y_pred ))

              precision    recall  f1-score   support

        -328       0.00      0.00      0.00         0
        -326       0.00      0.00      0.00         1
        -324       0.00      0.00      0.00         1
        -316       0.00      0.00      0.00         1
        -314       0.00      0.00      0.00         1
        -298       0.00      0.00      0.00         2
        -244       0.00      0.00      0.00         1
        -241       0.00      0.00      0.00         1
        -231       0.00      0.00      0.00         1
        -222       0.00      0.00      0.00         1
        -214       0.00      0.00      0.00         1
        -213       0.00      0.00      0.00         1
        -196       0.00      0.00      0.00         1
        -186       0.00      0.00      0.00         1
        -185       0.33      1.00      0.50         1
        -183       0.00      0.00      0.00         1
        -177       0.00      0.00      0.00         1
        -169       0.00    

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### RandomForestClassifier

In [21]:
random_forest = RandomForestClassifier(criterion = "gini",  max_depth= 6, n_estimators=100, max_features='log2',)

random_forest.fit(train, y_train)
Y_pred = random_forest.predict(test)
print(classification_report(y_test,Y_pred ))

              precision    recall  f1-score   support

        -328       0.00      0.00      0.00         0
        -326       0.00      0.00      0.00         1
        -324       0.00      0.00      0.00         1
        -316       0.00      0.00      0.00         1
        -314       0.00      0.00      0.00         1
        -307       0.00      0.00      0.00         0
        -298       0.00      0.00      0.00         2
        -244       0.00      0.00      0.00         1
        -241       0.00      0.00      0.00         1
        -240       0.00      0.00      0.00         0
        -239       0.00      0.00      0.00         0
        -231       0.00      0.00      0.00         1
        -222       0.00      0.00      0.00         1
        -214       0.25      1.00      0.40         1
        -213       0.00      0.00      0.00         1
        -200       0.00      0.00      0.00         0
        -196       0.00      0.00      0.00         1
        -186       0.00    

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Bagging classifier Decision tree

In [22]:
bag_clf_DT=BaggingClassifier(DecisionTreeClassifier(),n_estimators=100,max_samples=100,bootstrap=True,n_jobs=-1)
bag_clf_DT.fit(train, y_train)
Y_pred = bag_clf_DT.predict(test)
print(classification_report(y_test,Y_pred ))

              precision    recall  f1-score   support

        -328       0.00      0.00      0.00         0
        -326       0.00      0.00      0.00         1
        -324       0.00      0.00      0.00         1
        -316       0.00      0.00      0.00         1
        -314       0.00      0.00      0.00         1
        -307       0.00      0.00      0.00         0
        -298       0.00      0.00      0.00         2
        -244       0.00      0.00      0.00         1
        -241       0.00      0.00      0.00         1
        -239       0.00      0.00      0.00         0
        -231       0.00      0.00      0.00         1
        -222       0.00      0.00      0.00         1
        -214       0.20      1.00      0.33         1
        -213       0.00      0.00      0.00         1
        -204       0.00      0.00      0.00         0
        -196       0.00      0.00      0.00         1
        -186       0.00      0.00      0.00         1
        -185       1.00    

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Bagging classifier Gaussian NB


In [24]:
bag_clf_NB=BaggingClassifier(GaussianNB() ,n_estimators=100,max_samples=100,bootstrap=True,n_jobs=-1)
bag_clf_NB.fit(train, y_train)
Y_pred = bag_clf_NB.predict(test)
print(classification_report(y_test,Y_pred ))

ZeroDivisionError: ignored

### Ada Boost Classifier

In [25]:
Ada_boost = AdaBoostClassifier( n_estimators= 400, learning_rate= 0.05, algorithm= 'SAMME.R')                         
Ada_boost.fit(train, y_train)
Y_pred = Ada_boost.predict(test)
print(classification_report(y_test,Y_pred ))

              precision    recall  f1-score   support

        -326       0.00      0.00      0.00         1
        -324       0.00      0.00      0.00         1
        -316       0.00      0.00      0.00         1
        -314       0.00      0.00      0.00         1
        -298       0.00      0.00      0.00         2
        -244       0.00      0.00      0.00         1
        -241       0.00      0.00      0.00         1
        -231       0.00      0.00      0.00         1
        -222       0.00      0.00      0.00         1
        -214       0.00      0.00      0.00         1
        -213       0.00      0.00      0.00         1
        -196       0.00      0.00      0.00         1
        -186       0.00      0.00      0.00         1
        -185       0.00      0.00      0.00         1
        -183       0.00      0.00      0.00         1
        -177       0.00      0.00      0.00         1
        -169       0.00      0.00      0.00         1
        -164       0.00    

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Gradiant boosting with grid search cv 


In [26]:
gradientBoosting = GradientBoostingClassifier(min_samples_leaf= 40, n_estimators= 20, min_samples_split= 200, learning_rate= 1, max_depth= 7)                                       
gradientBoosting.fit(train, y_train)
Y_pred = gradientBoosting.predict(test)
print(classification_report(y_test,Y_pred ))

              precision    recall  f1-score   support

        -326       0.00      0.00      0.00         1
        -324       0.00      0.00      0.00         1
        -316       0.00      0.00      0.00         1
        -314       0.00      0.00      0.00         1
        -298       0.00      0.00      0.00         2
        -244       0.00      0.00      0.00         1
        -241       0.00      0.00      0.00         1
        -231       0.00      0.00      0.00         1
        -222       0.00      0.00      0.00         1
        -214       0.00      0.00      0.00         1
        -213       0.00      0.00      0.00         1
        -196       0.00      0.00      0.00         1
        -186       0.00      0.00      0.00         1
        -185       0.00      0.00      0.00         1
        -183       0.00      0.00      0.00         1
        -177       0.00      0.00      0.00         1
        -169       0.00      0.00      0.00         1
        -164       0.00    

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Naive Bayes

In [27]:
NB1 = GaussianNB()
NB1.fit(train, y_train)
Y_pred = NB1.predict(test)
print(classification_report(y_test,Y_pred ))

              precision    recall  f1-score   support

        -328       0.00      0.00      0.00         0
        -326       0.00      0.00      0.00         1
        -324       0.00      0.00      0.00         1
        -316       0.00      0.00      0.00         1
        -314       0.00      0.00      0.00         1
        -307       0.00      0.00      0.00         0
        -298       1.00      0.50      0.67         2
        -244       0.00      0.00      0.00         1
        -241       0.00      0.00      0.00         1
        -240       0.00      0.00      0.00         0
        -236       0.00      0.00      0.00         0
        -232       0.00      0.00      0.00         0
        -231       0.00      0.00      0.00         1
        -222       0.00      0.00      0.00         1
        -215       0.00      0.00      0.00         0
        -214       1.00      1.00      1.00         1
        -213       1.00      1.00      1.00         1
        -200       0.00    

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### MLP Classifier

In [28]:
from sklearn.neural_network import MLPClassifier 
mlp = MLPClassifier()
mlp.fit(train, y_train)
Y_pred = mlp.predict(test)
print(classification_report(y_test,Y_pred ))

              precision    recall  f1-score   support

        -357       0.00      0.00      0.00         0
        -328       0.00      0.00      0.00         0
        -326       0.00      0.00      0.00         1
        -324       0.00      0.00      0.00         1
        -316       0.00      0.00      0.00         1
        -314       0.00      0.00      0.00         1
        -307       0.00      0.00      0.00         0
        -298       0.00      0.00      0.00         2
        -244       0.00      0.00      0.00         1
        -241       0.00      0.00      0.00         1
        -240       0.00      0.00      0.00         0
        -236       0.00      0.00      0.00         0
        -232       0.00      0.00      0.00         0
        -231       0.00      0.00      0.00         1
        -222       0.00      0.00      0.00         1
        -214       0.50      1.00      0.67         1
        -213       0.00      0.00      0.00         1
        -204       0.00    

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Voting Classifier

In [ ]:
from sklearn.ensemble import VotingClassifier
voting_clf = VotingClassifier(
 estimators=[('lr', logreg), ('rf', random_forest), ('svc', svm_clf2), ('DT',decision_tree1),('NB',gaussian)],
 voting='hard'
 )
voting_clf.fit(train, y_train)
Y_pred = voting_clf.predict(test)
print(classification_report(y_test,Y_pred ))

NameError: ignored

In [ ]:
from sklearn.base import clone


class OrdinalClassifier():
    
    def __init__(self, clf):
        self.clf = clf
        self.clfs = {}
    
    def fit(self, X, y):
        self.unique_class = np.sort(np.unique(y))
        if self.unique_class.shape[0] > 2:
            for i in range(self.unique_class.shape[0]-1):
                # for each k - 1 ordinal value we fit a binary classification problem
                binary_y = (y > self.unique_class[i]).astype(np.uint8)
                clf = clone(self.clf)
                clf.fit(X, binary_y)
                self.clfs[i] = clf
    
    def predict_proba(self, X):
        clfs_predict = {k:self.clfs[k].predict_proba(X) for k in self.clfs}
        predicted = []
        for i,y in enumerate(self.unique_class):
            if i == 0:
                # V1 = 1 - Pr(y > V1)
                predicted.append(1 - clfs_predict[y][:,1])
            elif y in clfs_predict:
                # Vi = Pr(y > Vi-1) - Pr(y > Vi)
                 predicted.append(clfs_predict[y-1][:,1] - clfs_predict[y][:,1])
            else:
                # Vk = Pr(y > Vk-1)
                predicted.append(clfs_predict[y-1][:,1])
        return np.vstack(predicted).T
    
    def predict(self, X):
        return np.argmax(self.predict_proba(X), axis=1)

### Ordinal Regreesion

In [ ]:
from sklearn.linear_model import LinearRegression, LogisticRegression
from mord import LogisticAT
from mord import OrdinalRidge
class mord.OrdinalRidge(alpha=1.0, fit_intercept=True, normalize=False, copy_X=True, max_iter=None, tol=0.001, solver='auto')

logreg = LogisticAT()
logreg.fit(train, y_train)
Y_pred = logreg.predict(test)
print(classification_report(y_test,Y_pred ))
#http://rikunert.com/ordinal_rating
#https://statistics.laerd.com/spss-tutorials/ordinal-regression-using-spss-statistics.php